In [1]:
# ML Data Extraction Prototype
# This notebook will connect to the database, extract and join the tables needed
# to begin feature engineering for the next-race prediction model.

# 1. Imports and engine setup
import pandas as pd
from sqlalchemy import create_engine, text
from Data_Import.database import get_engine

# Create SQLAlchemy engine
engine = get_engine(echo=False)

ModuleNotFoundError: No module named 'Data_Import'

In [ ]:
# 2. Inspect available tables
with engine.connect() as conn:
    tables = conn.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")).fetchall()
tables_list = [t[0] for t in tables]
print("Available tables:", tables_list)

# 3. Load raw tables into DataFrames
# Athlete dimension
athletes_df = pd.read_sql_table('athlete', engine)
# Events dimension
events_df   = pd.read_sql_table('events', engine)
# Race results fact table
results_df  = pd.read_sql_table('race_results', engine)

print(f"Athletes: {len(athletes_df)} rows")
print(f"Events: {len(events_df)} rows")
print(f"Race results: {len(results_df)} rows")

# 4. Join tables for ML dataset
# Convert EventDate to datetime for feature engineering
events_df['EventDate'] = pd.to_datetime(events_df['EventDate'])
# Merge results with events and athletes
ml_df = (
    results_df
    .merge(events_df, on='EventID', how='left')
    .merge(athletes_df, left_on='athlete_id', right_on='athlete_id', how='left')
)

print(f"ML dataset shape: {ml_df.shape}")
ml_df.head()

# 5. Save or cache the extracted DataFrame for next steps
# e.g., to a parquet file
ml_df.to_parquet('data/ml_raw_dataset.parquet', index=False)
print("Raw ML dataset saved to data/ml_raw_dataset.parquet")
